<a href="https://colab.research.google.com/github/AntonioDeFA/etl_pyspark/blob/develop/Dados_IBGE_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark findspark
!pip install unidecode

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import udf
from pyspark.sql.functions import upper
from pyspark.sql.types import StringType
from unidecode import unidecode

In [16]:
def remove_accents(input_str):
    return unidecode(input_str)

In [ ]:
spark = SparkSession.builder \
    .appName("Join and Export to CSV") \
    .getOrCreate()

df_ibge = spark.read.csv('/content/cities-brazil-ibge.csv', header=True, inferSchema=True)
df_cases_covid = spark.read.csv('/content/cases-brazil-cities-time_2022.csv', header=True, inferSchema=True)

In [39]:
df_cases_covid_remove_stats = df_cases_covid.withColumn("city", split(df_cases_covid["city"], "/")[0])

In [40]:
remove_accents_udf = udf(remove_accents, StringType())
df_cases_covid_remove_accent = df_cases_covid_remove_stats.withColumn("city", remove_accents_udf(df_cases_covid_remove_stats["city"]))

In [41]:
df_cases_covid_upper = df_cases_covid_remove_accent.withColumn("city", upper(df_cases_covid_remove_accent["city"]))

In [42]:
joined_df_ibge_covid_cases = df_ibge.withColumnRenamed("LocalCidade", "city").join(df_cases_covid_upper, 'city', 'inner')


In [43]:
joined_df_ibge_covid_cases_translated = joined_df_ibge_covid_cases.withColumnRenamed("epi_week ", " epi_mes") \
.withColumnRenamed("date", "data") \
.withColumnRenamed("state", "estado") \
.withColumnRenamed("city", "cidade") \
.withColumnRenamed("newDeaths", "novas_mortes") \
.withColumnRenamed("deaths", "mortes") \
.withColumnRenamed("newCases", "novos_casos") \
.withColumnRenamed("totalCases", "total_casos") \
.withColumnRenamed("deaths_per_100k_inhabitants", "morte_por_100k_habitantes") \
.withColumnRenamed("totalCases_per_100k_inhabitants", "total_morte_por_100k_habitantes") \
.withColumnRenamed("deaths_by_totalCases", "mortes_por_total_de_casos")

In [44]:
joined_df_ibge_covid_cases_droped_columns = joined_df_ibge_covid_cases_translated.drop("epi_week","RegiaoBrasil")

In [47]:
joined_df_ibge_covid_cases_droped_columns.write.csv("relatorio-final.csv", header=True)

spark.stop()